In [3]:
import tqdm as tqdm
import torch
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms
from torch.nn.functional import relu
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from skimage import io
import torch
from skimage import color
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.init as init
from PIL import Image, ImageOps
import pickle

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('y')
else:
    device = torch.device('cpu')
    print('n')


y


In [6]:
images_path = '/vast/amr10211/cv_data/images' 
train_path = '../data/full_data.csv'
input_directory = '/vast/amr10211/cv_data/segm_preprocessed'
output_directory = '/vast/amr10211/cv_data/masked_images'

df = pd.read_csv(train_path)

png_files = [filename for filename in os.listdir(input_directory) if filename.endswith('.png')]

for file in tqdm.tqdm(png_files, desc="Masking PNG files"):
    segm_name = file
    
    img_name = df[df['segm_name'] == segm_name]['img_name'].iloc[0]
    
    image_filepath = os.path.join(images_path, img_name)
    segm_filepath = os.path.join(input_directory, segm_name)
    
    image = Image.open(image_filepath)
    segmentation = Image.open(segm_filepath)
    
    labels = [1,2,3,4,5,6]
    
    segmentation = np.array(segmentation)
    for l in labels:
        mask = np.where(segmentation == l, l, 0)
    
        if np.sum(mask) >= 5:
            mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
            masked_image = np.multiply(image, mask)
            
            filename, file_extension = os.path.splitext(file)
            output_filepath = filename + "_" + str(l) + file_extension
            output_filepath = os.path.join(output_directory, output_filepath)
            
            masked_image = Image.fromarray(masked_image.astype(np.uint8))
            masked_image.save(output_filepath)


Masking PNG files: 100%|██████████| 12701/12701 [25:45<00:00,  8.22it/s]


In [ ]:
class FashionDataset(Dataset):
    def __init__(self, csv_file, img_dir, segm_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file filename information.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.segm_dir = segm_dir
        self.transform = transform


    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir,
                                self.data_frame.iloc[idx, 1])

        segm_name = os.path.join(self.segm_dir,
                                self.data_frame.iloc[idx, -1])

        image = Image.open(img_name)
        label = Image.open(segm_name)


        #Convert to grayscale 
        # image = image.convert('L')
        
        #Resize image and label to 572x572
        image = image.resize((572, 572))
        label = label.resize((572, 572))

        #Crop label to match probability output from model
        crop = transforms.CenterCrop(388)
        label = crop(label)

        
        #Normalize the image
        image = np.asarray(image)
        min_image = np.min(image)
        max_image = np.max(image)
        image = (image - min_image)/(max_image - min_image + 1e-4)
        
        #Convert to tensors
        image = torch.tensor(np.array(image)).float().permute(2, 0, 1)
        label = torch.tensor(np.array(label)).long()

        #sample = {'x': , 'y': }
        sample = {'x': image, 'y': label}
    
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.resnet18 = models.resnet18(weights=None) 
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, 3)

    def forward(self, x):
        x = self.resnet18(x)
        return x